In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os.path as op

import matplotlib.pyplot as plt
from qgrid import show_grid
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tools.data_handling import enrich_emails, unique_recipients, address_book
from tools.data_cleansing import clean
from tools.features import split_tokenizer, stem_tokenizer, lemmatize_tokenizer
from tools.evaluation import top_emails, evaluate
from tools.training import data_generator, EvaluateAndSave

# Process data

In [ ]:
overwrite = False
df_emails = enrich_emails(overwrite=overwrite)

## Train / Test split

In [ ]:
ratio = 0.9
n_train = int(ratio * df_emails.shape[0])
df_train = df_emails.sample(n=n_train, random_state=0)
df_test = df_emails.drop(df_train.index)

## Clean & Tokenize

In [ ]:
recipients = unique_recipients(df_train)
names = address_book(recipients)
excepted_words = names.add("fyi")

Clean

In [ ]:
df_train["clean_body"] = clean(df_train["body"], excepted_words, only_english=True)
df_test["clean_body"] = clean(df_test["body"], excepted_words, only_english=True)

Tokenize

In [ ]:
input_bow = TfidfVectorizer(norm="l2", tokenizer=stem_tokenizer, min_df=5)
output_bow = CountVectorizer(tokenizer=split_tokenizer, vocabulary=recipients)

input_bow.fit(df_train["clean_body"])
output_bow.fit(df_train["recipients"])

n_features = len(input_bow.vocabulary_)
n_outputs = len(output_bow.vocabulary_)
print("Features: {} - Outputs: {}".format(n_features, n_outputs))

# Prediction

## Neural Network

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
batch_size = 32
samples_per_epoch = 32 * batch_size
nb_epoch = 20

inputs = Input(shape=(n_features,))
x = inputs
#x = Dense(n_features//2, activation='relu')(x)
predictions = Dense(n_outputs, activation='softmax')(x)

model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy')

In [ ]:
# Create callbacks
X_test = input_bow.transform(df_test["clean_body"]).toarray()
ground_truth = df_test["recipients"].str.split(expand=True).as_matrix()
recipients_map = np.array(output_bow.get_feature_names())

callbacks = []
filepath = "models/weights_{epoch:02d}.hdf5"
evaluate_and_save = EvaluateAndSave(X_test, recipients_map, ground_truth,
                                    batch_size=batch_size)
callbacks.append(evaluate_and_save)

generator = data_generator(
    df_train, input_bow, output_bow, batch_size=batch_size
    )

model.fit_generator(
    generator,
    samples_per_epoch=samples_per_epoch, nb_epoch=nb_epoch,
    callbacks=callbacks, nb_worker=1)